In [ ]:
# pip install scikit-learn
# !pip install tensorflow

In [1]:
import resource

# Get the current memory usage
memory_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

# Convert memory usage to a human-readable format (in megabytes)
memory_usage_mb = memory_usage / 1024

print(f"Memory Usage: {memory_usage_mb} MB")

Memory Usage: 63144.0 MB


In [1]:
import dash
from dash import Dash, dcc, html   
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LSTM, GRU, Reshape
# from sklearn.metrics import mean_squared_error
# import matplotlib.pyplot as plt

In [84]:
#Data overview
df = pd.read_csv('data.csv')
pre_predict = pd.read_csv('pre_predict.csv')
error = pd.read_csv('error.csv')

In [46]:
#Data lstm
predict_lstm = pd.read_csv('predict.csv')
loss_lstm=pd.read_csv('loss_lstm.csv')
train_datal_lstm = pd.read_csv('train_check.csv')
test_datal_lstm = pd.read_csv('test_check.csv')
predict_lstm = pd.read_csv('predict.csv')

In [58]:
#Data gru
predict_gru = pd.read_csv('predict_df_gru.csv')
loss_gru=pd.read_csv('loss_gru.csv')


In [ ]:
# Data model 3

In [48]:
test_data=df[len(train_datal_lstm):].values
test_data = pd.DataFrame(test_data, columns=['Date','price_coffee','price_oil'])


In [64]:
# data table 
data = predict_lstm 

In [110]:
# Khởi tạo ứng dụng Dash
app = dash.Dash(__name__)

# Thiết lập giao diện
app.layout = html.Div(className='container', children=[
    # Phần navigation bar bên trái
    html.Div(className='navbar', children=[
        dcc.Dropdown(
            id='model-dropdown',
            options=[
                {'label': 'Model Overview', 'value': 'MO'},
                {'label': 'LSTM model', 'value': 'LSTM'},
                {'label': 'GRU model', 'value': 'GRU'},
                {'label': 'XGBOOST model', 'value': 'XGBOOST'}
            ],
            value='MO',  # Giá trị mặc định
            style={'text-align': 'center'}
        )
    ]),

    # Phần hiển thị biểu đồ chia thành 4 ô
    html.Div(className='display', children=[
        # Ô trên bên trái
        html.Div(id='chart1', className='chart', children=[
        ]),
        # Ô trên bên phải
        html.Div(id='chart2', className='chart', children=[
        ]),
        # Ô dưới bên trái
        html.Div(id='chart3', className='chart', children=[
        ]),
        # Ô dưới bên phải
        html.Div(id='chart4', className='chart', children=[
        ]),
    ], style={'display': 'grid', 'grid-template-columns': '1fr 1fr', 'grid-template-rows': '1fr 1fr'}),
])

# Callback để cập nhật biểu đồ dựa trên giá trị của dropdown
@app.callback(
    dash.dependencies.Output('chart1', 'children'),
    dash.dependencies.Output('chart2', 'children'),
    dash.dependencies.Output('chart3', 'children'),
    dash.dependencies.Output('chart4', 'children'),
    dash.dependencies.Input('model-dropdown', 'value')
)
def update_charts(model):
    if model == 'LSTM':
        return (
            # Ô trên bên trái
        html.Div(className='chart', children=[
            dcc.Graph(
                figure=go.Figure(data=[
                    go.Scatter(x=loss_lstm['Epoch'],y=loss_lstm['Loss'], mode='lines', name='loss epoch'),
                ], layout=go.Layout(
                    title={
                        'text': 'Loss per epoch line chart',
                        'x': 0.5,  # Giữa theo chiều ngang
                        'xanchor': 'center',
                        'y': 0.90  # Đặt ở phía trên
                    },
                    xaxis={'title': 'epochs'},
                    yaxis={'title': 'loss'}
                )),
                style={'display': 'flex', 'justify-content': 'center'}
            )
        ]),
        # Ô trên bên phải
        html.Div(className='chart', children=[
            dcc.Graph(
                figure=go.Figure(data=[
                    go.Scatter(x=df['Date'], y=df['price_coffee'], mode='lines', name='Price Coffee'),
                    go.Scatter(x=train_datal_lstm['Date'], y=train_datal_lstm['prediction'], mode='lines', name='Train Data'),
                    go.Scatter(x=test_datal_lstm['Date'], y=test_datal_lstm['prediction'], mode='lines', name='Test Data')
                ], layout=go.Layout(
                    title={
                        'text': 'Price chart of coffee train and test',
                        'x': 0.5,  # Giữa theo chiều ngang
                        'xanchor': 'center',
                        'y': 0.90  # Đặt ở phía trên
                    },
                    xaxis={'title': 'Years'},
                    yaxis={'title': 'Price'}
                )),
                style={'display': 'flex', 'justify-content': 'center'}
            )
        ]),
        # Ô dưới bên trái
        html.Div(className='chart', children=[
            dcc.Graph(
                figure=go.Figure(data=[
                    go.Scatter(x=test_data['Date'], y=test_data['price_coffee'], mode='lines', name='Old Price'),
                    go.Scatter(x=predict_lstm['Date'], y=predict_lstm['price_coffee_pre'], mode='lines', name='Predict LSTM'),
                ], layout=go.Layout(
                    title={
                        'text': 'Price chart of predict Coffee price ',
                        'x': 0.5,  # Giữa theo chiều ngang
                        'xanchor': 'center',
                        'y': 0.90  # Đặt ở phía trên
                    },
                    xaxis={'title': 'Years'},
                    yaxis={'title': 'Price'}
                )),
                style={'display': 'flex', 'justify-content': 'center'}
            )
        ]),
        # Ô dưới bên phải
        html.Div(
                className='chart',    style={'width': '76.5%', 'padding-left':'80px', }, children=[
                    html.H3('Price Coffee Prediction'),  # Tiêu đề bảng
                    dash_table.DataTable(
                    columns=[{"name": col, "id": col} for col in predict_lstm.columns],
                    data=predict_lstm.to_dict('records'),
                    page_size=10,
                    style_table={'overflowX': 'auto'},  # Hiển thị thanh cuộn ngang khi nội dung quá rộng
                    style_cell={'textAlign': 'center'},  # Căn trái dòng dữ liệu
                    style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}],  # Tạo sự tương phản giữa các dòng
                    page_action='native',  # Chuyển trang thông qua phía máy khách (client-side paging)
                )

            ]),
        )
    elif model == 'GRU':
        return (
            html.Div(className='chart', children=[
            dcc.Graph(
                figure=go.Figure(data=[
                    go.Scatter(x=loss_gru['Epoch'],y=loss_gru['Loss'], mode='lines', name='loss epoch'),
                ], layout=go.Layout(
                    title={
                        'text': 'Loss per epoch line chart',
                        'x': 0.5,  # Giữa theo chiều ngang
                        'xanchor': 'center',
                        'y': 0.90  # Đặt ở phía trên
                    },
                    xaxis={'title': 'epochs'},
                    yaxis={'title': 'loss'}
                )),
                style={'display': 'flex', 'justify-content': 'center'}
            )
        ]),
        # Ô trên bên phải
        html.Div(className='chart', children=[
            dcc.Graph(
                figure=go.Figure(data=[
                    go.Scatter(x=df['Date'], y=df['price_coffee'], mode='lines', name='Price Coffee'),
                    go.Scatter(x=train_datal_lstm['Date'], y=train_datal_lstm['prediction'], mode='lines', name='Train Data'),
                    go.Scatter(x=test_datal_lstm['Date'], y=test_datal_lstm['prediction'], mode='lines', name='Test Data')
                ], layout=go.Layout(
                    title={
                        'text': 'Price chart of coffee train and test',
                        'x': 0.5,  # Giữa theo chiều ngang
                        'xanchor': 'center',
                        'y': 0.90  # Đặt ở phía trên
                    },
                    xaxis={'title': 'Years'},
                    yaxis={'title': 'Price'}
                )),
                style={'display': 'flex', 'justify-content': 'center'}
            )
        ]),
        # Ô dưới bên trái
        html.Div(className='chart', children=[
            dcc.Graph(
                figure=go.Figure(data=[
                    go.Scatter(x=test_data['Date'], y=test_data['price_coffee'], mode='lines', name='Old Price'),
                    go.Scatter(x=predict_gru['Date'], y=predict_gru['predict_gru'], mode='lines', name='Predict GRU'),
                ], layout=go.Layout(
                    title={
                        'text': 'Price chart of predict Coffee price ',
                        'x': 0.5,  # Giữa theo chiều ngang
                        'xanchor': 'center',
                        'y': 0.90  # Đặt ở phía trên
                    },
                    xaxis={'title': 'Years'},
                    yaxis={'title': 'Price'}
                )),
                style={'display': 'flex', 'justify-content': 'center'}
            )
        ]),
        # Ô dưới bên phải
        html.Div(
                className='chart',    style={'width': '76.5%', 'padding-left':'80px', }, children=[
                    html.H3('Price Coffee Prediction'),  # Tiêu đề bảng
                    dash_table.DataTable(
                    columns=[{"name": col, "id": col} for col in predict_gru.columns],
                    data=predict_gru.to_dict('records'),
                    page_size=10,
                    style_table={'overflowX': 'auto'},  # Hiển thị thanh cuộn ngang khi nội dung quá rộng
                    style_cell={'textAlign': 'center'},  # Căn trái dòng dữ liệu
                    style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}],  # Tạo sự tương phản giữa các dòng
                    page_action='native',  # Chuyển trang thông qua phía máy khách (client-side paging)
                )

            ]),
        )
    elif model == 'XGBOOST':
        return (
            # Biểu đồ 1
            html.Div(),
            # Biểu đồ 2
            html.Div(),
            # Biểu đồ 3
            html.Div(),
            # Biểu đồ 4
            html.Div(
                className='chart',children=[
                    dash_table.DataTable(
                        
                    columns=[{"name": col, "id": col} for col in data.columns],
                    data=data.to_dict('records'),
                    page_size=10,
                    style_table={'overflowX': 'auto'},  # Hiển thị thanh cuộn ngang khi nội dung quá rộng
                    style_cell={'textAlign': 'center'},  # Căn trái dòng dữ liệu
                    style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}],  # Tạo sự tương phản giữa các dòng
                    page_action='native',  # Chuyển trang thông qua phía máy khách (client-side paging)
                )

            ]),
        )
    else:
        return (
            # Ô trên bên trái
        html.Div(className='chart', children=[
            dcc.Graph(
                figure=go.Figure(data=[
                    go.Scatter(x=df['Date'], y=df['price_coffee'], mode='lines', name='Price Coffee'),
                    go.Scatter(x=df['Date'], y=df['price_oil'], mode='lines', name='Price Oil')
                ], layout=go.Layout(
                    title={
                        'text': 'Coffee and oil price chart',
                        'x': 0.5,  # Giữa theo chiều ngang
                        'xanchor': 'center',
                        'y': 0.90  # Đặt ở phía trên
                    },
                    xaxis={'title': 'Years'},
                    yaxis={'title': 'Price'}
                )),
                style={'display': 'flex', 'justify-content': 'center'}
            )
        ]),
        # Ô trên bên phải
        html.Div(className='chart', children=[
            dcc.Graph(
                figure=go.Figure(
                    data=[
                        go.Bar(name='LSTM Model', x=error['calculate'], y=error['lstm']),
                        go.Bar(name='Gru Model', x=error['calculate'], y=error['gru']),
                        go.Bar(name='Xgboost Model', x=error['calculate'], y=error['xgboost'])
                    ],
                    layout=go.Layout(
                        title={
                            'text': 'Bar chart error level',
                            'x': 0.5,
                            'y': 0.90
                        },
                        xaxis={'title': 'Method'},
                        yaxis={'title': 'Number'},
                        barmode='group'  # Hiển thị cột kề nhau
                    )
                ),
                style={'display': 'flex', 'justify-content': 'center'}
            )        
        ]),
        # Ô dưới bên trái
        html.Div(className='chart', children=[
            dcc.Graph(
                figure=go.Figure(data=[
                    go.Scatter(x=test_data['Date'], y=test_data['price_coffee'], mode='lines', name='Old Price'),
                    # go.Scatter(x=pre_predict['Date'], y=pre_predict['price_coffee'], mode='lines', name='Old Price'),
                    go.Scatter(x=predict_lstm['Date'], y=predict_lstm['price_coffee_pre'], mode='lines', name='Predict LSTM'),
                    go.Scatter(x=predict_gru['Date'], y=predict_gru['predict_gru'], mode='lines', name='Predict Gru')
                ], layout=go.Layout(
                    title={
                        'text': 'Price chart of predict Coffee price ',
                        'x': 0.5,  # Giữa theo chiều ngang
                        'xanchor': 'center',
                        'y': 0.90  # Đặt ở phía trên
                    },
                    xaxis={'title': 'Years'},
                    yaxis={'title': 'Price'}
                )),
                style={'display': 'flex', 'justify-content': 'center'}
            )
        ]),
        # Ô dưới bên phải
        html.Div(className='chart', children=[
            dcc.Graph(
                figure=go.Figure(
                    data=[
                        go.Bar(name='LSTM Model', x=error['calculate'], y=error['lstm']),
                        go.Bar(name='Gru Model', x=error['calculate'], y=error['gru']),
                        go.Bar(name='Xgboost Model', x=error['calculate'], y=error['xgboost'])
                    ],
                    layout=go.Layout(
                        title={
                            'text': 'Bar chart error level',
                            'x': 0.5,
                            'y': 0.90
                        },
                        xaxis={'title': 'Method'},
                        yaxis={'title': 'Number'},
                        barmode='group'  # Hiển thị cột kề nhau
                    )
                ),
                style={'display': 'flex', 'justify-content': 'center'}
            )        
        ]),
        )

# Khởi chạy ứng dụng
if __name__ == '__main__':
    app.run_server(debug=True)
